<a href="https://colab.research.google.com/github/limshaocong/SysBERT/blob/main/MLM_TAPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminaries

Install HuggingFace transformers and datasets libaries.

In [1]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 7.4 MB/s 
     |████████████████████████████████| 4.2 MB 76.4 MB/s 
     |████████████████████████████████| 212 kB 82.4 MB/s 
     |████████████████████████████████| 140 kB 79.4 MB/s 
     |████████████████████████████████| 1.1 MB 50.4 MB/s 
     |████████████████████████████████| 86 kB 6.9 MB/s 
     |████████████████████████████████| 86 kB 6.3 MB/s 
     |████████████████████████████████| 596 kB 87.7 MB/s 
     |████████████████████████████████| 127 kB 96.5 MB/s 
     |████████████████████████████████| 6.6 MB 78.6 MB/s 
     |████████████████████████████████| 144 kB 92.5 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 271 kB 90.7 MB/s 
     |████████████████████████████████| 112 kB 91.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling

In [2]:
! nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-7a39b033-6d78-827a-6453-cf6c04b9db26)


# Import and Pre-process Data


In [3]:
! git clone https://github.com/limshaocong/SysBERT/

Cloning into 'SysBERT'...
remote: Enumerating objects: 523, done.
remote: Counting objects: 100% (523/523), done.
remote: Compressing objects: 100% (489/489), done.
remote: Total 523 (delta 69), reused 435 (delta 22), pack-reused 0
Receiving objects: 100% (523/523), 6.68 MiB | 8.53 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [4]:
from datasets import load_dataset

train_path = '/content/SysBERT/Data/mlm_train.csv'
test_path = '/content/SysBERT/Data/mlm_test.csv'

ds = load_dataset(
    'csv',
    data_files = {
        'train' : train_path,
        'test' : test_path
    }
)

ds

Using custom data configuration default-d768fafae18c56eb


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d768fafae18c56eb/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['source', 'reqs'],
        num_rows: 37797
    })
    test: Dataset({
        features: ['source', 'reqs'],
        num_rows: 1990
    })
})

In [15]:
from transformers import AutoTokenizer

# model_checkpoint = 'bert-base-cased'
model_checkpoint = 'roberta-base'

tokenizer = AutoTokenizer.from_pretrained(
    model_checkpoint,
    use_fast = True
)

def encode(requirements):
    return tokenizer(requirements['reqs'], truncation = False)

ds_tokenized = ds.map(
    encode,
    batched = True,
    remove_columns = ['source', 'reqs']
)

ds_tokenized

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

  0%|          | 0/38 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (733 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/2 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 37797
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1990
    })
})

In [16]:
chunk_size = 128

def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result['labels'] = result['input_ids'].copy()
    
    return result

In [17]:
ds_chunked = ds_tokenized.map(group_texts, batched = True)
ds_chunked

  0%|          | 0/38 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 12204
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 650
    })
})

In [18]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer = tokenizer,
    mlm_probability = 0.15
)

In [19]:
batch_size = 16

batched_train_ds = ds_chunked['train'].to_tf_dataset(
    columns = ['input_ids', 'attention_mask', 'labels'],
    collate_fn = data_collator,
    shuffle = True,
    batch_size = batch_size,
)

batched_val_ds = ds_chunked['test'].to_tf_dataset(
    columns = ['input_ids', 'attention_mask', 'labels'],
    collate_fn = data_collator,
    shuffle = False,
    batch_size = batch_size,
)

# Task-Adpative Pre-Training (TAPT)

In [20]:
import tensorflow as tf
from transformers import TFAutoModelForMaskedLM, create_optimizer

tf.keras.utils.set_random_seed(1)
num_epochs = 100

def create_model():
  
  model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)

  num_train_steps = len(batched_train_ds) * num_epochs

  optimizer, schedule = create_optimizer(
      init_lr = 2e-5,
      num_warmup_steps = 1000,
      num_train_steps = num_train_steps,
      weight_decay_rate = 0.01,
  )

  model.compile(optimizer = optimizer)

  return model

model = create_model()

model.summary()

Downloading:   0%|          | 0.00/627M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForMaskedLM.

All the layers of TFRobertaForMaskedLM were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForMaskedLM for predictions without further training.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour, please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


Model: "tf_roberta_for_masked_lm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLayer  multiple                 124055040 
 )                                                               
                                                                 
 lm_head (TFRobertaLMHead)   multiple                  39642969  
                                                                 
Total params: 124,697,433
Trainable params: 124,697,433
Non-trainable params: 0
_________________________________________________________________


In [21]:
import math
import os
from tensorflow.keras.callbacks import Callback, CSVLogger, ModelCheckpoint
from transformers import PushToHubCallback

class Perplexity(Callback):

    def __init__(self, validation):
        super(Perplexity, self).__init__()
        self.validation = validation

    def on_epoch_end(self, epoch, logs = {}):

        val_score = self.model.evaluate(self.validation)     
        val_perplexity = math.exp(val_score)
        logs['val_perplexity'] = val_perplexity

perplexity_cb = Perplexity(batched_val_ds)

csvlogger_cb = CSVLogger('results.csv')

checkpoint_path = 'model_checkpoints/reqbert-mlm-epoch{epoch}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

modelcheckpoint_cb = ModelCheckpoint(
    filepath = checkpoint_path,
    save_weights_only = True,
    verbose = 1
)

In [22]:
callbacks = [perplexity_cb, csvlogger_cb, modelcheckpoint_cb]

In [23]:
model.fit(
    batched_train_ds,
    validation_data = batched_val_ds,
    epochs = num_epochs,
    callbacks = callbacks
)

Epoch 1/100
41/41 [==============================] - 5s 122ms/step - loss: 2.0439

Epoch 1: saving model to model_checkpoints/reqbert-mlm-epoch1.ckpt
762/762 [==============================] - 292s 361ms/step - loss: 2.4126 - val_loss: 1.9932 - val_perplexity: 7.7205
Epoch 2/100
41/41 [==============================] - 5s 121ms/step - loss: 1.8710

Epoch 2: saving model to model_checkpoints/reqbert-mlm-epoch2.ckpt
762/762 [==============================] - 272s 357ms/step - loss: 2.0656 - val_loss: 1.8643 - val_perplexity: 6.4950
Epoch 3/100
41/41 [==============================] - 5s 121ms/step - loss: 1.7879

Epoch 3: saving model to model_checkpoints/reqbert-mlm-epoch3.ckpt
762/762 [==============================] - 272s 357ms/step - loss: 1.9265 - val_loss: 1.7710 - val_perplexity: 5.9767
Epoch 4/100
41/41 [==============================] - 5s 121ms/step - loss: 1.7021

Epoch 4: saving model to model_checkpoints/reqbert-mlm-epoch4.ckpt
762/762 [==============================] - 272

KeyboardInterrupt: ignored

In [ ]:
# Get the latest checkpoint weights
os.listdir(checkpoint_dir)
latest = tf.train.latest_checkpoint(checkpoint_dir)

# Reinstate checkpoint weights
model = create_model()
model.load_weights(latest)

# Re-evaluate model on validation data
model.evaluate(batched_val_ds)